In [ ]:
from pyspark.sql.functions import *

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 3, Finished, Available, Finished)

In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
SILVER_TABLES_BASE = (
    "abfss://cad6716d-54df-4655-aed7-d883bc9a65b4@onelake.dfs.fabric.microsoft.com/"
    "2aec1ba9-3225-4830-8516-53d0668f4551/Tables/"
)

df   = spark.read.format("delta").load(SILVER_TABLES_BASE + "silver_customers")
df_o = spark.read.format("delta").load(SILVER_TABLES_BASE + "silver_orders")
df_p = spark.read.format("delta").load(SILVER_TABLES_BASE + "silver_products")
df_e = spark.read.format("delta").load(SILVER_TABLES_BASE + "silver_events")
df_t = spark.read.format("delta").load(SILVER_TABLES_BASE + "silver_tickets")


StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 4, Finished, Available, Finished)

In [2]:
display(df.limit(5))
display(df_t.limit(5))
display(df_e.limit(5))
display(df_p.limit(5))
display(df_o.limit(5))

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 42a9e55d-3567-46ec-b869-a8a9b64d5abd)

SynapseWidget(Synapse.DataFrame, 8b0ed628-a1ec-4d61-906e-9c4ae10530ba)

SynapseWidget(Synapse.DataFrame, 422bc33d-6565-47cc-8db4-339f36f2beb9)

SynapseWidget(Synapse.DataFrame, 378bb9b1-129c-4794-a480-4b75471082e5)

SynapseWidget(Synapse.DataFrame, 9842c91a-905a-4e86-9642-53f317791ae5)

In [ ]:
df_o.printSchema()

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 6, Finished, Available, Finished)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_amount: decimal(10,2) (nullable = true)
 |-- order_date: date (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- status: string (nullable = true)
 |-- quantity: integer (nullable = true)



In [4]:
df_t.count()

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 7, Finished, Available, Finished)

30000

In [ ]:

orders_agg = (
    df_o
    .groupBy("customer_id")
    .agg(
        count("order_id").alias("total_orders"),
        sum("order_amount").alias("total_spend"),
        avg("order_amount").alias("avg_order_value"),
        max("order_date").alias("last_order_date"),
        sum("quantity").alias("total_quantity")
    )
)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 8, Finished, Available, Finished)

In [ ]:
display(orders_agg)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 346ba8c8-b265-4a4e-861f-f67516e8bab2)

In [ ]:
tickets_agg = (
    df_t
    .groupBy("customer_id")
    .agg(
        count("ticket_id").alias("total_tickets"),
        avg("resolution_time_hours").alias("avg_resolution_time_hours"),
        max("ticket_created").alias("last_ticket_date")
    )
)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 10, Finished, Available, Finished)

In [ ]:
display(tickets_agg)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5d41e503-139e-4bac-8d12-1dacebd1b88e)

In [ ]:
events_agg = (
    df_e
    .where("customer_id IS NOT NULL")
    .groupBy("customer_id")
    .agg(
        count("event_id").alias("total_events"),
        countDistinct("session_id").alias("total_sessions"),
        max("timestamp").alias("last_active_time")
    )
)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 12, Finished, Available, Finished)

In [ ]:
display(events_agg)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6a116d34-09b9-4e0d-9626-75b1806731dd)

In [ ]:
gold_customer_360 = (
    df
    .join(orders_agg,  "customer_id", "left")
    .join(tickets_agg, "customer_id", "left")
    .join(events_agg,  "customer_id", "left")
    
)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 14, Finished, Available, Finished)

In [ ]:
gold_customer_360 = gold_customer_360.withColumn( "last_order_date", when(col("last_order_date") > current_date(), None) .otherwise(col("last_order_date")) )
gold_customer_360 = gold_customer_360.withColumn( "last_active_time", when(col("last_active_time") > current_date(), None) .otherwise(col("last_active_time")) )

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 15, Finished, Available, Finished)

In [ ]:
gold_customer_360=gold_customer_360.withColumn("days_since_last_order", datediff(current_date(), col("last_order_date")))
gold_customer_360=gold_customer_360.withColumn("days_since_last_active", datediff(current_date(), col("last_active_time")))

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 16, Finished, Available, Finished)

In [ ]:
display(gold_customer_360)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cb41e66a-e161-4d92-b3b1-92031b3d696f)

In [ ]:
gold_customer_360 = gold_customer_360.fillna({
    "total_orders": 0,
    "total_spend": 0.0,
    "avg_order_value": 0.0,
    "total_quantity": 0,
    "total_tickets": 0,
    "avg_resolution_time_hours": 0.0,
    "total_events": 0,
    "total_sessions": 0
})

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 18, Finished, Available, Finished)

In [ ]:
df_o.select(
    min("order_date").alias("min_order_date"),
    max("order_date").alias("max_order_date")
).show()

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 19, Finished, Available, Finished)

+--------------+--------------+
|min_order_date|max_order_date|
+--------------+--------------+
|    2023-12-03|    2027-01-06|
+--------------+--------------+



In [ ]:
df_e.select(
    min("timestamp").alias("min_event_date"),
    max("timestamp").alias("max_event_date")
).show()

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 20, Finished, Available, Finished)

+--------------+--------------+
|min_event_date|max_event_date|
+--------------+--------------+
|    2025-09-02|    2028-02-10|
+--------------+--------------+



In [ ]:
df_t.select(
    min("ticket_created").alias("min_ticket_date"),
    max("ticket_created").alias("max_ticket_date")
).show()

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 21, Finished, Available, Finished)

+---------------+---------------+
|min_ticket_date|max_ticket_date|
+---------------+---------------+
|     2023-12-03|     2027-01-06|
+---------------+---------------+



In [ ]:
gold_customer_360.printSchema()

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 22, Finished, Available, Finished)

root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- signup_date: date (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- device_ids: string (nullable = true)
 |-- source: string (nullable = true)
 |-- total_orders: long (nullable = false)
 |-- total_spend: decimal(20,2) (nullable = true)
 |-- avg_order_value: decimal(14,6) (nullable = true)
 |-- last_order_date: date (nullable = true)
 |-- total_quantity: long (nullable = false)
 |-- total_tickets: long (nullable = false)
 |-- avg_resolution_time_hours: double (nullable = false)
 |-- last_ticket_date: date (nullable = true)
 |-- total_events: long (nullable = false)
 |-- total_sessio

In [ ]:
display(gold_customer_360)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e1f4a595-c41a-4e0b-a791-e85940e7dafc)

In [ ]:
from pyspark.sql.functions import datediff, current_date, floor

gold_customer_360 = gold_customer_360.withColumn(
    "age",
    floor(datediff(current_date(), col("dob")) / 365)
)


StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 24, Finished, Available, Finished)

In [ ]:
gold_customer_360 = gold_customer_360.withColumn(
    "age_group",
    when(col("age") < 25, "Under 25")
    .when(col("age") <= 35, "25–35")
    .when(col("age") <= 45, "36–45")
    .when(col("age") <= 55, "46–55")
    .otherwise("55+")
)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 25, Finished, Available, Finished)

In [ ]:
display(gold_customer_360)

StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2f0d8191-b2b7-460f-a67c-85b410828229)

In [ ]:
gold_customer_360.write.mode("overwrite").format("delta").saveAsTable("gold_customer_360")


StatementMeta(, afd800a5-f7cb-4720-88a1-d9791c75a2f0, 27, Finished, Available, Finished)